In [1]:
# Imports
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('..')

import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import MENDER

from utils.data_loader import DataLoader

In [2]:
# Load MERSCOPE Dataset from SODB
loader = DataLoader("Dataset13_MS_raw")
adata_dict = loader.load()
adata = list(adata_dict.values())[0]

Loading dataset: Dataset13_MS_raw
load experiment[Dataset13] in dataset[Dataset13_MS_raw]
Dataset loaded successfully.


In [3]:
# Check available batches and their slices
adata.obs[['batch', 'slice_id']].drop_duplicates()


,batch,slice_id
110883424764611924400221639916314253469-0,0,R1S1
277373730858255322904479591336292143718-1,1,R2S1
139968683432966769265787739231843442191-2,2,R3S1
149164679103246548309819743981609972453-3,3,R1S2
100442548580636641738686294721955425236-4,4,R2S2
158338042824236264719696604356349910479-5,5,R3S2
156852667528872626811117292962470921390-6,6,R1S3
222213390088484216253925626300058690969-7,7,R2S3
102664563492900048462363937849459428087-8,8,R3S3


In [4]:
# To avoid batch effect we focus on a single slice from the dataset
adata_slice = adata[adata.obs['slice_id'] == 'R1S1'].copy()

In [5]:
from MENDER import compute_PAS, compute_CHAOS
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Select slice R1S1 to avoid batch effects
adata_slice = adata[adata.obs['slice_id'] == 'R1S1'].copy()

# Preprocess for PCA
adata_pca = adata_slice.copy()
sc.pp.highly_variable_genes(adata_pca, flavor="seurat_v3", n_top_genes=4000)
sc.pp.normalize_total(adata_pca, inplace=True)
sc.pp.log1p(adata_pca)
sc.pp.pca(adata_pca)
sc.pp.neighbors(adata_pca)
sc.tl.leiden(adata_pca, resolution=2.0, key_added='ct_pca')
adata_pca.obs['ct_pca'] = adata_pca.obs['ct_pca'].astype('category')

print("=== PCA Evaluation ===")
ari_pca = adjusted_rand_score(adata_pca.obs['ct'], adata_pca.obs['ct_pca'])
nmi_pca = normalized_mutual_info_score(adata_pca.obs['ct'], adata_pca.obs['ct_pca'])
print(f"Ground Truth ARI (PCA): {ari_pca:.3f}")
print(f"Ground Truth NMI (PCA): {nmi_pca:.3f}")

# Run MENDER on PCA results
msm_pca = MENDER.MENDER_single(adata_pca, ct_obs='ct_pca')
msm_pca.set_MENDER_para(n_scales=2, nn_mode='ring', nn_para=6)
msm_pca.run_representation()
msm_pca.run_clustering_normal(-0.2)

# MENDER metrics (PCA)
nmi_dom_pca = normalized_mutual_info_score(msm_pca.adata_MENDER.obs['ct_pca'], msm_pca.adata_MENDER.obs['MENDER'])
pas_pca = compute_PAS(msm_pca.adata_MENDER, pred_key='MENDER')
chaos_pca = compute_CHAOS(msm_pca.adata_MENDER, pred_key='MENDER')

print("--- PCA MENDER Evaluation ---")
print(f"Spatial NMI: {nmi_dom_pca:.3f}")
print(f"PAS: {pas_pca:.3f}")
print(f"CHAOS: {chaos_pca:.3f}")
print()

# UMAP comparisons
n_components_list = [10, 20, 30, 40, 50]

for n_comp in n_components_list:
    print(f"=== UMAP with {n_comp} components ===")
    
    adata_umap = adata_slice.copy()
    sc.pp.highly_variable_genes(adata_umap, flavor="seurat_v3", n_top_genes=4000)
    sc.pp.normalize_total(adata_umap, inplace=True)
    sc.pp.log1p(adata_umap)
    sc.pp.pca(adata_umap, n_comps=n_comp)
    sc.pp.neighbors(adata_umap, n_pcs=n_comp)
    sc.tl.umap(adata_umap)

    ct_key = f'ct_umap_{n_comp}'
    sc.tl.leiden(adata_umap, resolution=2.0, key_added=ct_key)
    adata_umap.obs[ct_key] = adata_umap.obs[ct_key].astype('category')

    # Compare clustering to ground truth
    ari = adjusted_rand_score(adata_umap.obs['ct'], adata_umap.obs[ct_key])
    nmi = normalized_mutual_info_score(adata_umap.obs['ct'], adata_umap.obs[ct_key])
    print(f"Ground Truth ARI: {ari:.3f}")
    print(f"Ground Truth NMI: {nmi:.3f}")

    # Run MENDER on UMAP
    msm = MENDER.MENDER_single(adata_umap, ct_obs=ct_key)
    msm.set_MENDER_para(n_scales=2, nn_mode='ring', nn_para=6)
    msm.run_representation()
    msm.run_clustering_normal(-0.2)

    # Domain Evaluation
    msm.adata_MENDER.obs[ct_key] = adata_umap.obs[ct_key]
    nmi_dom = normalized_mutual_info_score(msm.adata_MENDER.obs[ct_key], msm.adata_MENDER.obs['MENDER'])
    pas = compute_PAS(msm.adata_MENDER, pred_key='MENDER')
    chaos = compute_CHAOS(msm.adata_MENDER, pred_key='MENDER')

    print("--- MENDER Evaluation ---")
    print(f"Spatial NMI: {nmi_dom:.3f}")
    print(f"PAS: {pas:.3f}")
    print(f"CHAOS: {chaos:.3f}")
    print()


=== PCA Evaluation ===
Ground Truth ARI (PCA): 0.762
Ground Truth NMI (PCA): 0.841
scale 0, median #cells per ring (r=6): 6.0
scale 1, median #cells per ring (r=6): 7.0
--- PCA MENDER Evaluation ---
Spatial NMI: 0.408
PAS: 0.065
CHAOS: 0.010

=== UMAP with 10 components ===
Ground Truth ARI: 0.612
Ground Truth NMI: 0.737
scale 0, median #cells per ring (r=6): 6.0
scale 1, median #cells per ring (r=6): 7.0
--- MENDER Evaluation ---
Spatial NMI: 0.350
PAS: 0.079
CHAOS: 0.010

=== UMAP with 20 components ===
Ground Truth ARI: 0.682
Ground Truth NMI: 0.811
scale 0, median #cells per ring (r=6): 6.0
scale 1, median #cells per ring (r=6): 7.0
--- MENDER Evaluation ---
Spatial NMI: 0.405
PAS: 0.067
CHAOS: 0.010

=== UMAP with 30 components ===
Ground Truth ARI: 0.737
Ground Truth NMI: 0.830
scale 0, median #cells per ring (r=6): 6.0
scale 1, median #cells per ring (r=6): 7.0
--- MENDER Evaluation ---
Spatial NMI: 0.415
PAS: 0.075
CHAOS: 0.010

=== UMAP with 40 components ===
Ground Truth ARI: